In [1]:
import numpy as np
import pandas as pd

## Data preprocessing


In [2]:
df = pd.read_csv('Boston.csv')

In [3]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
df.isna().sum()

crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
black      0
lstat      0
medv       0
dtype: int64

In [5]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [6]:
# normal the data
from sklearn.preprocessing import MinMaxScaler

# initize a scaler
scaler_X = MinMaxScaler()

# fit and transform
X_scaled = scaler_X.fit_transform(X)


In [7]:
# split the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=0)

## Gradient

In [35]:
class WEN():
    '''
    Implement weighted elastic net
    '''
    
    def __init__(self, l1= 0.5, l2= 0.5, weight =  None,
                 step_size = 1e-3, max_iter = 50, 
                 tol= 1e-4, random_state = 0):
        '''
        Args:
            l1 (float): A float between 0 and 1 for lasso regularization
            l2 (float): A float between 0 and 1 for riage regularization
            weight (array): A 2D weight with the suitable dimensions. Default is identify matrix
            step_size (float): A float that determines the step size
            max_iter (int): The maximun number of iterations
            tol (float): The tolerlance for the solution
            random_state (int): The random state
            
        Returns: None
        '''
        self._weight = weight
        self._max_iter = max_iter
        self._step_size = step_size
        self._tol = tol
        self._random_state = random_state
        self._coeff = None
        self._num_obersvations = None
        self._num_features = None
        self._max_peak = None
        self._min_peak = None
        self._l1 = l1
        self._l2 = l2
        self._gradient = None
        self._cost = None
        self._response = None
                
    def _get_num_observations(self, X):
        '''
        Args:
            X (array): A matrix that contains data
        
        Returns: None
        '''
        self._num_obersvations = (np.array(X)).shape[0]
        
    def _get_num_features(self, X):
        '''
        Args:
            X (array): A matrix that contains data
            
        Returns: None
        '''
        self._num_features = (np.array(X)).shape[1]
    
    def _get_peaks(self, X):
        '''
        Args:
            X (array): A matrix that contains data
            
        Returns: None
        '''
        self._max_peak = np.max(np.array(X), axis = 0)
        self._min_peak = np.min(np.array(X), axis = 0)
    
    def _get_initial_coeff(self, X):
        '''
        Args:
            X (array): A matrix that contians data
            
        Reutrns: None
        '''
        np.random_state = self._random_state
        
        if self._num_features is None:
            self._get_num_features(X)
        if (self._min_peak is None) or (self._max_peak is None):
            self._get_peaks(X)
        ranges = (self._max_peak - self._min_peak).reshape(self._num_features,1)
        self._coeff = ranges*np.random.random(size = (self._num_features, 1)) \
            + self._min_peak.reshape(self._num_features, 1)
    
    def _get_initial_weight(self, X):
        '''
        Args:
            X (array): A matrix that contains data
        
        Returns: None 
        '''
        if self._num_obersvations is None:
            self._get_num_observations(X)
        if self._weight is None:
            self._weight = np.identity(self._num_obersvations)
            
        
    def _should_stop(self, old_coeff, iters):
        '''
        Args:
            old_coeff (array): The old coefficients 
            iters: The number of iterations
        
        Returns:
            boolean: True for meeting stop conditions. Otherwise returns False
        '''
        if iters > self._max_iter:
            return True
        else:
            if old_coeff is None:
                return False
            else:
                return (np.abs(old_coeff - self._coeff).sum() < self._tol)
            
    def _OLS_coeff(self, X, y):
        '''
        Args:
            X (array): A matrix that contians data
            y (array): A array that contains labels
            
        Returns: 
            array: a array with the same size of self._coeff
        '''
        y = np.array(y).reshape(self._num_obersvations, 1)
        X = np.array(X)
        return np.linalg.inv(np.transpose(X)@X)@np.transpose(X)@y
        
    def _calculate_gradient(self, X, y):
        '''
        Args:
            X (array): A matrix that contians data
            y (array): A array that contains labels
            
        Returns: 
            array: a array with the same size of self._coeff
        '''
        y = np.array(y).reshape(self._num_obersvations, 1)
        X = np.array(X)
        sub_derivative = self._subgradient_absfunction(self._coeff)
        I = np.identity(self._num_features)
        
        g1 = 2*(np.transpose(X)@self._weight@X + self._l2*I)@self._coeff/(1 + self._l2)
        g2 = -2*np.transpose(X)@self._weight@y
        g3 = self._l1*sub_derivative
        
        self._gradient = g1 + g2 + g3

    def _calculate_cost(self, X, y):
        '''
        Args:
            X (array): A matrix that contians data
            y (array): A array that contains labels
            
        Returns: 
            float : a float represents the loss
        '''
        
        y = np.array(y).reshape(self._num_obersvations, 1)
        X = np.array(X)
        I = np.identity(self._num_features)
        
        lf1 = np.transpose(self._coeff)@(np.transpose(X)@self._weight@X + self._l2*I)/(1+ self._l2)@self._coeff
        lf2 = -2*np.transpose(y)@self._weight@X@self._coeff 
        lf3 = self._l1*np.linalg.norm(self._coeff, ord=1)
        
        self._cost = np.asscalar(lf1 + lf2)+ lf3
    
    def _calculate_coeff(self, X, y, gradient):
        '''
        Args:
            X (array): A matrix that contians data
            y (array): A array that contains labels
            
        '''
        self._coeff = self._coeff - self._step_size*gradient
    
    def _calculate_response(self, X):
        '''
        Args:
            X (array): A matrix that contians data
        '''
        self._response = np.array(X)@self._coeff
        
        
    def _calculate_weight(self, X, y):
        '''
        Args:
            X (array): A matrix that contians data
            y (array): A array that contains labels
        '''
        self._calculate_response(X)
        y = np.array(y).reshape(self._num_obersvations, 1)
        difference = 1/(y - self._response)**2
        return np.diag(difference.flatten())
    
    @staticmethod
    def _subgradient_absfunction(x):
        '''
        Args: None
            
        Returns: 
            array: a array with the same size of self._coeff
        '''
        sub_derivative = (x != 0)*np.sign(x) # when component != 0
        sub_derivative += (x == 0)*(2*np.random.random() - 1) # when component == 0
        return sub_derivative  
    
    def fit(self, X, y):
        '''
        Args:
            X (array): array that contains data
            y (array): array that contains labels
    
        '''
        iters = 0
        best_coeff = None
        best_cost = None
        old_coeff = None
        old_cost = None
                
        self._get_initial_coeff(X)
        self._get_initial_weight(X)
                
        while not self._should_stop(old_coeff, iters):
            iters += 1
            old_coeff = self._coeff
            old_cost = self._cost
            
            self._calculate_gradient(X, y)
            self._calculate_cost(X, y)
            self._calculate_coeff(X, y, self._gradient)
            
            if best_cost is None:
                best_cost = self._cost
                best_coeff = self._coeff
            else:
                if self._cost < best_cost:
                    best_cost = self._cost
                    best_coeff = self._coeff
             
            print(iters, old_cost, best_cost, np.max(self._weight))
        

In [36]:
net1 = WEN(step_size = 1e-3, max_iter = 5000)

In [37]:
net1.fit(X_train, y_train)

1 None -50282.8889754 1.0
2 -50282.8889754 -228138.582851 1.0
3 -228138.582851 -250556.667416 1.0
4 -250556.667416 -258719.162201 1.0
5 -258719.162201 -264182.255656 1.0
6 -264182.255656 -268267.373699 1.0
7 -268267.373699 -271385.358019 1.0
8 -271385.358019 -273794.956828 1.0
9 -273794.956828 -275683.107692 1.0
10 -275683.107692 -277184.974986 1.0
11 -277184.974986 -278399.479169 1.0
12 -278399.479169 -279399.08997 1.0
13 -279399.08997 -280236.835689 1.0
14 -280236.835689 -280951.715666 1.0
15 -280951.715666 -281572.674181 1.0
16 -281572.674181 -282120.869931 1.0
17 -282120.869931 -282611.950049 1.0
18 -282611.950049 -283057.528038 1.0
19 -283057.528038 -283466.263148 1.0
20 -283466.263148 -283844.648796 1.0
21 -283844.648796 -284197.58858 1.0
22 -284197.58858 -284528.817214 1.0
23 -284528.817214 -284841.013019 1.0
24 -284841.013019 -285136.329874 1.0
25 -285136.329874 -285416.837763 1.0
26 -285416.837763 -285683.950762 1.0
27 -285683.950762 -285938.820393 1.0
28 -285938.820393 -28618

243 -293391.082021 -293393.038666 1.0
244 -293393.038666 -293394.97022 1.0
245 -293394.97022 -293396.877095 1.0
246 -293396.877095 -293398.759695 1.0
247 -293398.759695 -293400.618415 1.0
248 -293400.618415 -293402.453645 1.0
249 -293402.453645 -293404.265763 1.0
250 -293404.265763 -293406.055143 1.0
251 -293406.055143 -293407.82215 1.0
252 -293407.82215 -293409.567145 1.0
253 -293409.567145 -293411.290477 1.0
254 -293411.290477 -293412.992493 1.0
255 -293412.992493 -293414.67353 1.0
256 -293414.67353 -293416.333921 1.0
257 -293416.333921 -293417.973991 1.0
258 -293417.973991 -293419.594059 1.0
259 -293419.594059 -293421.194439 1.0
260 -293421.194439 -293422.775438 1.0
261 -293422.775438 -293424.337359 1.0
262 -293424.337359 -293425.880496 1.0
263 -293425.880496 -293427.40514 1.0
264 -293427.40514 -293428.911576 1.0
265 -293428.911576 -293430.400083 1.0
266 -293430.400083 -293431.870937 1.0
267 -293431.870937 -293433.324406 1.0
268 -293433.324406 -293434.760755 1.0
269 -293434.760755 -

488 -293559.891187 -293560.084287 1.0
489 -293560.084287 -293560.275958 1.0
490 -293560.275958 -293560.46621 1.0
491 -293560.46621 -293560.655055 1.0
492 -293560.655055 -293560.842506 1.0
493 -293560.842506 -293561.028574 1.0
494 -293561.028574 -293561.213271 1.0
495 -293561.213271 -293561.396607 1.0
496 -293561.396607 -293561.578595 1.0
497 -293561.578595 -293561.759246 1.0
498 -293561.759246 -293561.93857 1.0
499 -293561.93857 -293562.116579 1.0
500 -293562.116579 -293562.293283 1.0
501 -293562.293283 -293562.468694 1.0
502 -293562.468694 -293562.642822 1.0
503 -293562.642822 -293562.815678 1.0
504 -293562.815678 -293562.987272 1.0
505 -293562.987272 -293563.157614 1.0
506 -293563.157614 -293563.326716 1.0
507 -293563.326716 -293563.494586 1.0
508 -293563.494586 -293563.661235 1.0
509 -293563.661235 -293563.826674 1.0
510 -293563.826674 -293563.990911 1.0
511 -293563.990911 -293564.153957 1.0
512 -293564.153957 -293564.315822 1.0
513 -293564.315822 -293564.476514 1.0
514 -293564.4765

732 -293582.452814 -293582.489277 1.0
733 -293582.489277 -293582.525506 1.0
734 -293582.525506 -293582.561504 1.0
735 -293582.561504 -293582.597271 1.0
736 -293582.597271 -293582.63281 1.0
737 -293582.63281 -293582.668121 1.0
738 -293582.668121 -293582.703207 1.0
739 -293582.703207 -293582.738069 1.0
740 -293582.738069 -293582.772708 1.0
741 -293582.772708 -293582.807126 1.0
742 -293582.807126 -293582.841325 1.0
743 -293582.841325 -293582.875305 1.0
744 -293582.875305 -293582.909069 1.0
745 -293582.909069 -293582.942618 1.0
746 -293582.942618 -293582.975952 1.0
747 -293582.975952 -293583.009075 1.0
748 -293583.009075 -293583.041986 1.0
749 -293583.041986 -293583.074689 1.0
750 -293583.074689 -293583.107183 1.0
751 -293583.107183 -293583.13947 1.0
752 -293583.13947 -293583.171552 1.0
753 -293583.171552 -293583.20343 1.0
754 -293583.20343 -293583.235106 1.0
755 -293583.235106 -293583.26658 1.0
756 -293583.26658 -293583.297854 1.0
757 -293583.297854 -293583.32893 1.0
758 -293583.32893 -29

988 -293587.109784 -293587.117252 1.0
989 -293587.117252 -293587.124675 1.0
990 -293587.124675 -293587.132054 1.0
991 -293587.132054 -293587.139389 1.0
992 -293587.139389 -293587.146679 1.0
993 -293587.146679 -293587.153926 1.0
994 -293587.153926 -293587.161129 1.0
995 -293587.161129 -293587.168289 1.0
996 -293587.168289 -293587.175407 1.0
997 -293587.175407 -293587.182481 1.0
998 -293587.182481 -293587.189513 1.0
999 -293587.189513 -293587.196503 1.0
1000 -293587.196503 -293587.203451 1.0
1001 -293587.203451 -293587.210358 1.0
1002 -293587.210358 -293587.217223 1.0
1003 -293587.217223 -293587.224047 1.0
1004 -293587.224047 -293587.23083 1.0
1005 -293587.23083 -293587.237572 1.0
1006 -293587.237572 -293587.244275 1.0
1007 -293587.244275 -293587.250937 1.0
1008 -293587.250937 -293587.257559 1.0
1009 -293587.257559 -293587.264141 1.0
1010 -293587.264141 -293587.270685 1.0
1011 -293587.270685 -293587.277189 1.0
1012 -293587.277189 -293587.283654 1.0
1013 -293587.283654 -293587.29008 1.0
1

1245 -293588.094875 -293588.096504 1.0
1246 -293588.096504 -293588.098124 1.0
1247 -293588.098124 -293588.099735 1.0
1248 -293588.099735 -293588.101336 1.0
1249 -293588.101336 -293588.102928 1.0
1250 -293588.102928 -293588.10451 1.0
1251 -293588.10451 -293588.106084 1.0
1252 -293588.106084 -293588.107648 1.0
1253 -293588.107648 -293588.109203 1.0
1254 -293588.109203 -293588.110749 1.0
1255 -293588.110749 -293588.112286 1.0
1256 -293588.112286 -293588.113814 1.0
1257 -293588.113814 -293588.115333 1.0
1258 -293588.115333 -293588.116843 1.0
1259 -293588.116843 -293588.118344 1.0
1260 -293588.118344 -293588.119837 1.0
1261 -293588.119837 -293588.121321 1.0
1262 -293588.121321 -293588.122796 1.0
1263 -293588.122796 -293588.124263 1.0
1264 -293588.124263 -293588.125721 1.0
1265 -293588.125721 -293588.127171 1.0
1266 -293588.127171 -293588.128612 1.0
1267 -293588.128612 -293588.130045 1.0
1268 -293588.130045 -293588.131469 1.0
1269 -293588.131469 -293588.132886 1.0
1270 -293588.132886 -293588

1502 -293588.312624 -293588.31299 1.0
1503 -293588.31299 -293588.313354 1.0
1504 -293588.313354 -293588.313715 1.0
1505 -293588.313715 -293588.314075 1.0
1506 -293588.314075 -293588.314433 1.0
1507 -293588.314433 -293588.314788 1.0
1508 -293588.314788 -293588.315142 1.0
1509 -293588.315142 -293588.315493 1.0
1510 -293588.315493 -293588.315843 1.0
1511 -293588.315843 -293588.31619 1.0
1512 -293588.31619 -293588.316535 1.0
1513 -293588.316535 -293588.316879 1.0
1514 -293588.316879 -293588.31722 1.0
1515 -293588.31722 -293588.31756 1.0
1516 -293588.31756 -293588.317897 1.0
1517 -293588.317897 -293588.318233 1.0
1518 -293588.318233 -293588.318566 1.0
1519 -293588.318566 -293588.318898 1.0
1520 -293588.318898 -293588.319228 1.0
1521 -293588.319228 -293588.319556 1.0
1522 -293588.319556 -293588.319882 1.0
1523 -293588.319882 -293588.320206 1.0
1524 -293588.320206 -293588.320528 1.0
1525 -293588.320528 -293588.320849 1.0
1526 -293588.320849 -293588.321167 1.0
1527 -293588.321167 -293588.32148

1754 -293588.361361 -293588.361447 1.0
1755 -293588.361447 -293588.361532 1.0
1756 -293588.361532 -293588.361616 1.0
1757 -293588.361616 -293588.3617 1.0
1758 -293588.3617 -293588.361784 1.0
1759 -293588.361784 -293588.361867 1.0
1760 -293588.361867 -293588.36195 1.0
1761 -293588.36195 -293588.362032 1.0
1762 -293588.362032 -293588.362114 1.0
1763 -293588.362114 -293588.362195 1.0
1764 -293588.362195 -293588.362276 1.0
1765 -293588.362276 -293588.362356 1.0
1766 -293588.362356 -293588.362436 1.0
1767 -293588.362436 -293588.362515 1.0
1768 -293588.362515 -293588.362594 1.0
1769 -293588.362594 -293588.362673 1.0
1770 -293588.362673 -293588.362751 1.0
1771 -293588.362751 -293588.362829 1.0
1772 -293588.362829 -293588.362906 1.0
1773 -293588.362906 -293588.362982 1.0
1774 -293588.362982 -293588.363059 1.0
1775 -293588.363059 -293588.363134 1.0
1776 -293588.363134 -293588.36321 1.0
1777 -293588.36321 -293588.363285 1.0
1778 -293588.363285 -293588.363359 1.0
1779 -293588.363359 -293588.36343

2011 -293588.37288 -293588.3729 1.0
2012 -293588.3729 -293588.372919 1.0
2013 -293588.372919 -293588.372938 1.0
2014 -293588.372938 -293588.372958 1.0
2015 -293588.372958 -293588.372977 1.0
2016 -293588.372977 -293588.372996 1.0
2017 -293588.372996 -293588.373014 1.0
2018 -293588.373014 -293588.373033 1.0
2019 -293588.373033 -293588.373052 1.0
2020 -293588.373052 -293588.37307 1.0
2021 -293588.37307 -293588.373089 1.0
2022 -293588.373089 -293588.373107 1.0
2023 -293588.373107 -293588.373125 1.0
2024 -293588.373125 -293588.373144 1.0
2025 -293588.373144 -293588.373162 1.0
2026 -293588.373162 -293588.373179 1.0
2027 -293588.373179 -293588.373197 1.0
2028 -293588.373197 -293588.373215 1.0
2029 -293588.373215 -293588.373233 1.0
2030 -293588.373233 -293588.37325 1.0
2031 -293588.37325 -293588.373268 1.0
2032 -293588.373268 -293588.373285 1.0
2033 -293588.373285 -293588.373302 1.0
2034 -293588.373302 -293588.373319 1.0
2035 -293588.373319 -293588.373336 1.0
2036 -293588.373336 -293588.373353

2286 -293588.375588 -293588.375592 1.0
2287 -293588.375592 -293588.375596 1.0
2288 -293588.375596 -293588.3756 1.0
2289 -293588.3756 -293588.375604 1.0
2290 -293588.375604 -293588.375608 1.0
2291 -293588.375608 -293588.375612 1.0
2292 -293588.375612 -293588.375616 1.0
2293 -293588.375616 -293588.37562 1.0
2294 -293588.37562 -293588.375623 1.0
2295 -293588.375623 -293588.375627 1.0
2296 -293588.375627 -293588.375631 1.0
2297 -293588.375631 -293588.375635 1.0
2298 -293588.375635 -293588.375639 1.0
2299 -293588.375639 -293588.375642 1.0
2300 -293588.375642 -293588.375646 1.0
2301 -293588.375646 -293588.37565 1.0
2302 -293588.37565 -293588.375653 1.0
2303 -293588.375653 -293588.375657 1.0
2304 -293588.375657 -293588.375661 1.0
2305 -293588.375661 -293588.375664 1.0
2306 -293588.375664 -293588.375668 1.0
2307 -293588.375668 -293588.375671 1.0
2308 -293588.375671 -293588.375675 1.0
2309 -293588.375675 -293588.375678 1.0
2310 -293588.375678 -293588.375682 1.0
2311 -293588.375682 -293588.37568